# Setup

In [ ]:
import os

In [ ]:
run_in_colab = True

In [ ]:
if run_in_colab:
  !pip install transformers
  !pip install wandb
  !pip install git+https://github.com/google-research/bleurt.git
  !pip install datasets

  from google.colab import drive
  drive.mount('/content/drive')
  
  !git clone https://github.com/nofarmordehai/Learn-Chess-Commentary.git 'chess'
  CODE_DIR = 'chess'
  os.chdir(f'./{CODE_DIR}')

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import torch
import time
from Models.GPT2 import GPT2
from Models.BERT import BERT
from Dataset.MovesDataset import MovesDataset
from Configs.train_config import config

In [ ]:
import wandb

wandb.login()

In [ ]:
if run_in_colab:
  BASE_PATH = '/content/drive/MyDrive/NLP/'
else:
  BASE_PATH = '/home/joberant/nlp_fall_2021/nofarm/chess/'

In [ ]:
games_data_path = BASE_PATH + 'Data/NEW_attack/games_data'
saved_models_path = BASE_PATH + 'Models/'

# Model


In [ ]:
if config['Model'] == 'gpt2':
  gpt2 = GPT2()
  model = gpt2.model.train()
  tokenizer = gpt2.tokenizer
  max_length = 768

elif config['Model'] == 'bert-base':
  bert = BERT()
  model = bert.model.train()
  tokenizer = bert.tokenizer
  max_length = 512

# Dataset

In [ ]:
dataset = MovesDataset([f'{games_data_path}{i+1}.p' for i in range(config['NUMER_OF_DATA_DIRS'])], tokenizer, max_length=max_length) 

In [ ]:
train_size = int(config['train_precentege'] * len(dataset))
test_size = len(dataset) - train_size

In [ ]:
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=True)

# Train

Validation Text

In [ ]:
run = wandb.init(project="LmChess", config={'batch size': config['batch_size'], 'lr': config['lr'], 'epochs': config['epochs']})

In [ ]:
validation_proccessed_data, validation_attn_masks, validation_labels = next(iter(test_dataloader))

In [ ]:
validation_input_encodings = []
for i in range(config['batch_size']):
  textual_validation_data = tokenizer.decode(token_ids = validation_proccessed_data[i], skip_special_tokens=False).split('<comment>')

  validation_target_text = textual_validation_data[1].split('<|endoftext|>')[0]  # 'endoftext' for Bert
  validation_input_text = textual_validation_data[0] 

  wandb.log({f"validation_target_text {i}": wandb.Html(f'<p>{validation_target_text}</p>')})
  wandb.log({f"validation_input_text {i}": wandb.Html(f'<p>{validation_input_text}</p>')})

  comment_idx = list(validation_proccessed_data[i]).index(dataset.comment_encoding) + 1
  validation_input_encoding = validation_proccessed_data[i][:comment_idx].unsqueeze(0).cuda()
  #validation_input_encoding  = tokenizer.encode(validation_input_text, return_tensors="pt").cuda()
  
  validation_input_encodings.append(validation_input_encoding)

Train

In [ ]:
optimizer = AdamW(model.parameters(), lr= config['lr'])
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=5000, num_training_steps=-1
)

In [ ]:
loss = 0
pad_token_id = tokenizer('[PAD]')['input_ids'][0]

In [ ]:
epochs = config['epochs']

for epoch in range(epochs):
    with tqdm(total=len(train_dataset) / 2) as pbar:
        for idx,entry in enumerate(train_dataloader):

            if idx % 500 == 0 and idx != 0:
              for i in range(config['batch_size']):
                with torch.no_grad():
                    outputs = model.generate(validation_input_encodings[i], num_beams=2, no_repeat_ngram_size=2, max_length=max_length+1, pad_token_id=pad_token_id)
                    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
                wandb.log({f"output_text {i}": wandb.Html(f'<p>{output_text}</p>')})
            
            if idx % 50000 == 0:
              torch.save(model.state_dict(), f'{saved_models_path}{idx}_{time.time()}_{int(loss)}.bin')

            model.zero_grad()

            inputs = entry[0].cuda()
            attn_masks = entry[1].cuda()
            labels = entry[2].cuda()
            outputs = model(inputs, labels=labels, attention_mask = attn_masks)

            loss = outputs['loss']
            loss.backward()
            optimizer.step()
            scheduler.step()

            wandb.log({"epoch": epoch, "loss": loss})
            pbar.update(2)

# Evaluation

every t iterations calculate evaluation metrics for the current model and save the results

In [ ]:
from Evaluation.Metrics import perplexity, bleurt, bleu

In [ ]:
from Utils import get_targets_and_outputs

In [ ]:
model = model.eval()
# gpt2.load_model(model_path)

In [ ]:
test_perplexity = perplexity(model, test_dataloader)

In [ ]:
target_texts, output_texts = get_targets_and_outputs(gpt2, test_dataset, dataset.comment_encoding, pad_token_id)

In [ ]:
test_bleurt = bleurt(target_texts, output_texts)

In [ ]:
test_bleu = bleu(target_texts, output_texts)